In [7]:
!pip install streamlit

In [2]:
!pip install langchain pypdf chromadb sentence-transformers
!pip install -U langchain-community
!pip install langchain-openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [9]:
!streamlit run endeavor_ui.py

In [6]:
# Step 1: Allow user to upload a document
from IPython.display import display
import ipywidgets as widgets

# Create file upload widget
upload_widget = widgets.FileUpload(accept='.pdf,.docx,.txt', multiple=False)
display(upload_widget)

# Step 2: Get the uploaded filename and content
import io

def handle_upload(change):
    if upload_widget.value:
        uploaded_file_info = list(upload_widget.value.values())[0]
        uploaded_filename = uploaded_file_info['metadata']['name']
        uploaded_content = uploaded_file_info['content']

        # Optionally convert bytes to text (if .txt)
        if uploaded_filename.endswith('.txt'):
            content_text = uploaded_content.decode('utf-8')
            print("Text content preview:")
            print(content_text[:500])  # Preview first 500 chars

        print(f"\n✅ Uploaded file: {uploaded_filename}")
        globals()['uploaded_filename'] = uploaded_filename  # Store filename as a global variable

# Trigger function when upload is done
upload_widget.observe(handle_upload, names='value')

FileUpload(value=(), accept='.pdf,.docx,.txt', description='Upload')

#APi


In [5]:
import requests

url = "https://plankton-app-qajlk.ondigitalocean.app/extraction_api"
pdf_path = '/content/Hard - 2.pdf'

with open(pdf_path, "rb") as f:
    files = {'file': (pdf_path.split("/")[-1], f, 'application/pdf')}
    headers = {'accept': 'application/json'}

    response = requests.post(url, files=files, headers=headers)

# Output the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())

FileNotFoundError: [Errno 2] No such file or directory: '/content/Hard - 2.pdf'

In [ ]:
import requests
from typing import Optional, Dict, Any

def get_first_match(description: str, limit: int = 5) -> Optional[Dict[str, Any]]:
    """
    Query the match API with the given document description and return the first result.

    Args:
        description: The document description to match (may contain spaces, etc.).
        limit:       Number of results to request (default: 5).

    Returns:
        A dict with keys "match" and "score" for the top result, or None if no results.
    """
    url = "https://endeavor-interview-api-gzwki.ondigitalocean.app/match"
    params = {
        "query": description,
        "limit": limit
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()  # will raise an HTTPError if the request failed
    data = resp.json()
    if not isinstance(data, list) or len(data) == 0:
        return None
    return data[0]

# Example usage:
if _name_ == "_main_":
    desc = "Steel Nut M4 10mm Zinc Plated Coarse"
    top_hit = get_first_match(desc)
    if top_hit:
        print(f"Top match: {top_hit['match']} (score={top_hit['score']})")
    else:
        print("No matches found.")

NameError: name '_name_' is not defined

In [ ]:
import requests
import csv
import sys

def extract_pdf_to_csv(pdf_path: str, csv_path: str):
    url = "https://plankton-app-qajlk.ondigitalocean.app/extraction_api"
    with open(pdf_path, "rb") as f:
        files = {'file': (pdf_path.split("/")[-1], f, 'application/pdf')}
        headers = {'accept': 'application/json'}

        try:
            resp = requests.post(url, files=files, headers=headers, timeout=60)
            resp.raise_for_status()
        except requests.exceptions.HTTPError as err:
            # Try to read JSON error detail if present
            try:
                err_payload = resp.json()
                if isinstance(err_payload, dict) and "detail" in err_payload:
                    print(f"Extraction failed: {err_payload['detail']}", file=sys.stderr)
                else:
                    print(f"HTTP error: {err}", file=sys.stderr)
                    print("Response body:", resp.text, file=sys.stderr)
            except ValueError:
                print(f"HTTP error: {err}", file=sys.stderr)
                print("Response body:", resp.text, file=sys.stderr)
            return

        # parse JSON
        try:
            data = resp.json()
        except ValueError:
            print("❌ Response not valid JSON:", resp.text, file=sys.stderr)
            return

    # handle error payload vs. data payload
    if isinstance(data, dict) and "detail" in data:
        print(f"❌ Extraction error: {data['detail']}", file=sys.stderr)
        return

    if not isinstance(data, list) or not data or not isinstance(data[0], dict):
        print("❌ Unexpected response format, expected a list of dicts.", file=sys.stderr)
        print("Got:", data, file=sys.stderr)
        return
# write CSV
    with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

    print(f"✅ Successfully saved {len(data)} rows to {csv_path}")


extract_pdf_to_csv('/content/Hard - 2.pdf', '/content/output.csv')


✅ Successfully saved 2 rows to /content/output.csv


In [ ]:
import os
import io
import requests
from IPython.display import display
import ipywidgets as widgets
# from my_module import extract_pdf_to_csv  # Uncomment if implemented

# Upload widget
upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)
display(upload_widget)

# Global variable to store filename
uploaded_filename = None  # this will hold the filename for access later

def handle_upload(change):
    global uploaded_filename  # make filename accessible globally

    if upload_widget.value:
        uploaded_file = list(upload_widget.value.values())[0]
        uploaded_filename = uploaded_file['metadata']['name']
        content = uploaded_file['content']

        pdf_path = os.path.join("/content", uploaded_filename)
        csv_path = pdf_path.replace(".pdf", ".csv")

        with open(pdf_path, "wb") as f:
            f.write(content)
        print(f"✅ Saved uploaded PDF to: {pdf_path}")

        # extract_pdf_to_csv(pdf_path, csv_path)  # Uncomment if defined
        print(f"✅ CSV would be saved to: {csv_path}")

# Attach the upload handler
upload_widget.observe(handle_upload, names='value')


FileUpload(value={}, accept='.pdf', description='Upload')

✅ Saved uploaded PDF to: /content/Hard - 2.pdf
✅ CSV would be saved to: /content/Hard - 2.csv


In [ ]:
print("Uploaded filename was:", uploaded_filename)


Uploaded filename was: Hard - 2.pdf


0


In [ ]:
import requests

url = "https://plankton-app-qajlk.ondigitalocean.app/extraction_api"
pdf_path = uploaded_filename

with open(pdf_path, "rb") as f:
    files = {'file': (pdf_path.split("/")[-1], f, 'application/pdf')}
    headers = {'accept': 'application/json'}

    response = requests.post(url, files=files, headers=headers)

# Output the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())

Status Code: 200
Response JSON: [{'Line': 1, 'Product Code': 'zz_SODECK_7939', 'Manufacturer Code': 909854, 'Request Item': 'Titanium Washer M4 30mm Galvanized Fine', 'Quantity': '25 ea', 'Price': 90.866, 'Per': 'ea', 'Total': '2,271.65'}, {'Line': 2, 'Product Code': 'zz_SODECK_7940', 'Manufacturer Code': 852248, 'Request Item': 'Aluminum Stud M4 30mm Uncoated Coarse', 'Quantity': '8 ea', 'Price': 57.04, 'Per': 'ea', 'Total': '456.32'}]


In [ ]:
import json
def get_first_match(description: str, limit: int = 5) -> Optional[Dict[str, Any]]:
    match_url = "https://endeavor-interview-api-gzwki.ondigitalocean.app/match"
    params = {"query": description, "limit": limit}
    resp = requests.get(match_url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if not isinstance(data, list) or len(data) == 0:
        return None
    return data[0]


# Step 1: Extract JSON from API response
line_items = response.json()  # This is a list of dictionaries

# Step 2: Match each item and update the dictionary
for idx, item in enumerate(line_items):
    description = item.get("Request Item") or item.get("request item") or ""

    if not description:
        print(f"Item {idx+1}: No 'Request Item' found.")
        item["Matched Item"] = None
        item["Match Score"] = None
        continue

    print(f"\n🔍 Matching item {idx+1}: '{description}'")
    match = get_first_match(description)

    if match:
        item["Matched Item"] = match["match"]
        item["Match Score"] = match["score"]
        print(f"✅ Match: {match['match']} (Score: {match['score']})")
    else:
        item["Matched Item"] = None
        item["Match Score"] = None
        print("❌ No match found.")

# Step 3: Save the updated JSON (optional)
updated_json_path = "updated_line_items.json"
with open(updated_json_path, "w") as f:
    json.dump(line_items, f, indent=2)

print(f"\n✅ Updated JSON saved to: {updated_json_path}")


🔍 Matching item 1: 'Titanium Washer M4 30mm Galvanized Fine'
✅ Match: Titanium Washer M4 30mm Galvanized Fine (Score: 100.0)

🔍 Matching item 2: 'Aluminum Stud M4 30mm Uncoated Coarse'
✅ Match: Aluminum Stud M4 30mm Uncoated Coarse (Score: 100.0)

✅ Updated JSON saved to: updated_line_items.json


In [ ]:
import requests
from typing import Optional, Dict, Any
import ipywidgets as widgets
from IPython.display import display, clear_output

# 1. Define the match API call
def get_first_match(description: str, limit: int = 5) -> Optional[Dict[str, Any]]:
    match_url = "https://endeavor-interview-api-gzwki.ondigitalocean.app/match"
    params = {"query": description, "limit": limit}
    resp = requests.get(match_url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if not isinstance(data, list) or len(data) == 0:
        return None
    return data[0]

# 2. Store response from extraction API (already done before)
line_items = response.json()

# 3. Helper: Process each item one-by-one interactively
def process_items_interactively(items):
    index = 0  # Start from first item

    # Output widget to manage dynamic display
    out = widgets.Output()
    display(out)

    def show_next():
        nonlocal index  # Use nonlocal to modify the index from the outer function
        if index >= len(items):
            with out:
                clear_output()
                print("✅ All items processed.")
            return

        item = items[index]
        description = item.get("Request Item") or item.get("request item") or ""
        match_result = get_first_match(description)

        # Default to None in case of no match
        match_name = match_result["match"] if match_result else "No match found"
        match_score = match_result["score"] if match_result else None

        # UI elements
        yes_button = widgets.Button(description="Yes", button_style='success')
        no_button = widgets.Button(description="No", button_style='danger')
        corrected_input = widgets.Text(placeholder='Enter correct match here')
        submit_button = widgets.Button(description="Submit Correction", button_style='primary')
        correction_box = widgets.VBox([corrected_input, submit_button])
        correction_box.layout.visibility = 'hidden'  # Initially hidden

        with out:
            clear_output()
            print(f"🔍 Item {index+1}: {description}")
            print(f"💡 Suggested Match: {match_name} (Score: {match_score})")
            display(widgets.HBox([yes_button, no_button]), correction_box)

        # Button callbacks
        def handle_yes(b):
            nonlocal index  # Declare nonlocal for index in this function
            # Update the 'Request Item' with the suggested match
            item["Request Item"] = match_name
            item["Match Score"] = match_score
            index += 1
            show_next()

        def handle_no(b):
            # Show the correction input box when "No" is clicked
            correction_box.layout.visibility = 'visible'

        def handle_submit(b):
            nonlocal index  # Declare nonlocal for index in this function
            corrected = corrected_input.value.strip()
            if corrected:
                # Update 'Request Item' with the corrected value
                item["Request Item"] = corrected
                item["Match Score"] = "User Corrected"
                # After correction, update the item and proceed to the next one
                index += 1
                show_next()

        yes_button.on_click(handle_yes)
        no_button.on_click(handle_no)
        submit_button.on_click(handle_submit)

    # Start interaction
    show_next()

# 4. Call the function to start
process_items_interactively(line_items)

Output()

In [ ]:
import streamlit as st
import requests
import json
import csv
import os
from typing import Optional, Dict, Any

EXTRACTION_API_URL = "https://plankton-app-qajlk.ondigitalocean.app/extraction_api"
MATCH_API_URL = "https://endeavor-interview-api-gzwki.ondigitalocean.app/match"

def extract_pdf_content(file) -> Optional[list]:
    files = {'file': (file.name, file, 'application/pdf')}
    headers = {'accept': 'application/json'}
    try:
        response = requests.post(EXTRACTION_API_URL, files=files, headers=headers)
        response.raise_for_status()
        data = response.json()
        if isinstance(data, list):
            return data
        else:
            st.error("Unexpected response format from extraction API.")
            return None
    except requests.exceptions.RequestException as e:
        st.error(f"Error calling extraction API: {e}")
        return None

def get_first_match(description: str, limit: int = 5) -> Optional[Dict[str, Any]]:
    params = {"query": description, "limit": limit}
    try:
        response = requests.get(MATCH_API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        if isinstance(data, list) and len(data) > 0:
            return data[0]
        else:
            return None
    except requests.exceptions.RequestException as e:
        st.error(f"Error calling match API: {e}")
        return None

def save_csv(data, output_path="output.csv"):
    with open(output_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)
    return output_path

def main():
    st.title("📄 Endeavor AI - PDF Extractor and Matcher")
    uploaded_file = st.file_uploader("Upload a PDF file", type=["pdf"])

    if uploaded_file:
        with st.spinner("Extracting data from PDF..."):
            line_items = extract_pdf_content(uploaded_file)

        if line_items:
            st.success(f"✅ Extracted {len(line_items)} line items.")
            updated_items = []

            for idx, item in enumerate(line_items):
                st.markdown(f"### 🔍 Item {idx + 1}")
                description = item.get("Request Item") or item.get("request item") or ""
                st.write(f"**Description:** {description}")

                match_result = get_first_match(description)
                suggested = match_result["match"] if match_result else "No match found"
                score = match_result["score"] if match_result else None
                st.write(f"**Suggested Match:** {suggested}")
                st.write(f"**Score:** {score}")

                choice = st.radio(f"Accept suggestion for item {idx + 1}?", ["Yes", "No"], key=idx)
                if choice == "Yes":
                    item["Matched Item"] = suggested
                    item["Match Score"] = score
                else:
                    corrected = st.text_input(f"Enter corrected match for item {idx + 1}", key=f"corrected_{idx}")
                    if corrected:
                        item["Matched Item"] = corrected
                        item["Match Score"] = "User Corrected"
                    else:
                        item["Matched Item"] = None
                        item["Match Score"] = None

                updated_items.append(item)

            if st.button("💾 Save Output"):
                json_path = "updated_line_items.json"
                csv_path = "output.csv"

                with open(json_path, "w") as f:
                    json.dump(updated_items, f, indent=2)

                save_csv(updated_items, csv_path)
                st.success(f"✅ Saved results to `{json_path}` and `{csv_path}`")

                with open(json_path, "rb") as f:
                    st.download_button("Download JSON", f, file_name=json_path)

                with open(csv_path, "rb") as f:
                    st.download_button("Download CSV", f, file_name=csv_path)

if __name__ == "__main__":
    main()


2025-04-29 22:33:54.551 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 22:33:54.654 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-29 22:33:54.656 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 22:33:54.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 22:33:54.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 22:33:54.659 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 22:33:54.660 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 22:33:54.661 Thread 'MainThread': mi